# Sudoku

Aswin van Woudenberg (https://www.aswinvanwoudenberg.com | https://github.com/afvanwoudenberg)

In [2]:
from traitlets import Unicode, Bool, Int, List, validate, observe, TraitError, All
from ipywidgets import DOMWidget, register
import copy

@register
class Sudoku(DOMWidget):
    _view_name = Unicode('SudokuView').tag(sync=True)
    _view_module = Unicode('sudoku_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    # Attributes
    fixed = List(trait=Bool(), default_value=[False] * 81, minlen=81, maxlen=81, help="A list of booleans that indicate whether a value is part of the puzzle.").tag(sync=True)
    _value = List(trait=Int(), default_value=[0] * 81, minlen=81, maxlen=81, help="A list of integers for each cell.").tag(sync=True)
    disabled = Bool(False, help="Enable or disable user changes.").tag(sync=True)

    # Basic validator for value
    @validate('_value')
    def _valid_value(self, proposal):
        for i in proposal['value']:
            if i < 0 or i > 9:
                raise TraitError('Invalid value: all elements must be numbers from 0 to 9')
        return proposal['value']
    
    @property
    def value(self):
        return copy.deepcopy(self._value)
    
    @value.setter
    def value(self, v):
        self._value = v

    def __init__(self,*args,**kwargs):
        kwargs['_value'] = kwargs.pop('value', [0]*81)
        DOMWidget.__init__(self,*args,**kwargs)
    
    def __getitem__(self,index):
        return self._value[index]
    
    
    def __setitem__(self,index,val):
        vals = self.value
        vals[index] = val
        self._value = vals

In [3]:
%%javascript
require.undef('sudoku_widget');

define('sudoku_widget', ["@jupyter-widgets/base"], function(widgets) {
    
    // Define the SudokuView
    var SudokuView = widgets.DOMWidgetView.extend({
        
        // Render the view.
        render: function() {
            this.sudoku_table = document.createElement('table');
            this.sudoku_table.style.borderCollapse = 'collapse';
            this.sudoku_table.style.marginLeft = '0';
            
            for (let i=0; i<3; i++) {
                let colgroup = document.createElement('colgroup');
                colgroup.style.border = 'solid medium';
                for (let j=0; j<3; j++) {
                    let col = document.createElement('col');
                    col.style.border = 'solid thin';
                    col.style.width = '2em';
                    colgroup.appendChild(col);
                }
                this.sudoku_table.appendChild(colgroup);
            }
            
            for (let t=0; t<3; t++) {
                let tbody = document.createElement('tbody');
                tbody.style.border = 'solid medium';
                for (let r=0; r<3; r++) {
                    let tr = document.createElement('tr');
                    tr.style.height = '2em';
                    tr.style.border = 'solid thin';
                    for (let c=0; c<9; c++) {
                        let td = document.createElement('td');
                        tr.appendChild(td);
                    }
                    tbody.appendChild(tr);
                }
                this.sudoku_table.appendChild(tbody);
            }
            
            this.el.appendChild(this.sudoku_table);
            
            this.model_changed();
        
            // Python -> JavaScript update
            this.model.on('change', this.model_changed, this);
        },

        model_changed: function() {
            let tds = this.sudoku_table.getElementsByTagName('td');
            let disabled = this.model.get('disabled');
                        
            for (let i=0; i < 81; i++) {
                let td = tds[i];
                td.innerText = ''; // Delete td contents
                td.style.textAlign = 'center';
                td.style.height = '2em';
                let value = this.model.get('_value')[i];
                let fixed = this.model.get('fixed')[i];

                if (fixed && value > 0) {
                    let b = document.createElement('b');
                    b.innerText = value;
                    td.appendChild(b);
                } else if (disabled && value > 0) {
                    td.innerText = value;
                } else if (!disabled && !fixed) {
                    let input = document.createElement('input');
                    input.type = 'text';
                    input.maxLength = 1;
                    input.style.top = 0;
                    input.style.left = 0;
                    input.style.margin = 0;
                    input.style.height = '100%';
                    input.style.width = '100%';
                    input.style.border = 'none';
                    input.style.textAlign = 'center';
                    input.style.marginTop = 0;
                    input.style.padding = 0;
                    input.value = (value > 0 ? value : '');
                    input.oninput = this.input_input.bind(this, i);
                    input.onchange = this.input_changed.bind(this, i); // JavaScript -> Python update
                    td.appendChild(input);
                }
            }
            
        },
        
        input_input: function(i) {
            this.sudoku_table.getElementsByTagName('td')[i].getElementsByTagName('input')[0].value = 
                this.sudoku_table.getElementsByTagName('td')[i].
                    getElementsByTagName('input')[0].value.replace(/[^1-9]/g,'');
        },
        
        input_changed: function(i) {
            this.sudoku_table.getElementsByTagName('td')[i].getElementsByTagName('input')[0].value = 
                this.sudoku_table.getElementsByTagName('td')[i].
                    getElementsByTagName('input')[0].value.replace(/[^1-9]/g,'');
            let v = parseInt(this.sudoku_table.getElementsByTagName('td')[i].getElementsByTagName('input')[0].value) || 0;
            let value = this.model.get('_value').slice();
            value[i] = v;
            this.model.set('_value', value);
            this.model.save_changes();
        },
        
    });

    return {
        SudokuView: SudokuView
    }
    
});

<IPython.core.display.Javascript object>

In [4]:
import ipywidgets as widgets

In [5]:
puzzle1 = [
    0,8,5, 0,6,1, 0,0,0,
    9,0,4, 0,0,0, 0,0,0,
    0,0,0, 0,0,2, 3,0,8,
    
    0,4,0, 0,0,0, 0,0,2,
    7,0,0, 0,9,0, 5,0,0,
    0,0,0, 0,3,0, 8,0,0,
    
    0,0,0, 0,5,8, 0,0,0,
    0,0,0, 7,0,0, 0,1,0,
    6,0,0, 0,0,0, 0,0,4]

puzzle2 = [
    3,6,0, 0,0,0, 0,0,5,
    0,1,0, 0,9,0, 2,0,8,
    0,5,0, 1,8,0, 0,0,7,
    
    5,0,0, 0,0,6, 4,0,0,
    2,4,6, 0,5,0, 7,0,0,
    0,0,0, 0,7,0, 0,0,0,
    
    0,0,0, 0,0,7, 1,0,3,
    0,0,3, 9,4,0, 0,0,0,
    0,0,0, 0,0,1, 0,0,0]

puzzle3 = [
    0,2,0, 0,4,0, 0,0,5,
    0,5,8, 0,0,0, 0,0,0,
    0,1,0, 8,0,0, 4,0,0,
    
    7,0,0, 0,0,8, 0,4,0,
    0,0,1, 9,0,5, 7,0,0,
    0,3,0, 7,0,0, 0,0,2,
    
    0,0,4, 0,0,3, 0,1,0,
    0,0,0, 0,0,0, 9,6,0,
    2,0,0, 0,1,0, 0,5,0
]

sudoku = Sudoku(value=puzzle1, fixed=[v > 0 for v in puzzle1], disabled=False)
example_dropdown = widgets.Dropdown(
    options=[('Empty', [0] * 81), ('Example 1', puzzle1), ('Example 2', puzzle2), ('Example 3', puzzle3)], 
    value=puzzle1,
    layout=widgets.Layout(margin='10px 0px 0px 20px', width='150px')
)
solve_button = widgets.Button(
    description="Solve", 
    layout=widgets.Layout(margin='20px 0px 0px 20px', width='150px')
)
next_button = widgets.Button(
    description="Next", 
    layout=widgets.Layout(margin='20px 0px 0px 20px', width='150px', display='none')
)
vbox = widgets.VBox([example_dropdown, solve_button, next_button])
hbox = widgets.HBox([sudoku, vbox])
label = widgets.Label()

In [6]:
# global variables
gen = None
solution = None

def on_example_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        value = change['new']
        fixed = [v > 0 for v in value]
        sudoku.value = value
        sudoku.fixed = fixed
        label.value = ""
        solve_button.layout.display = 'inline-block'
        next_button.layout.display = 'none'

example_dropdown.observe(on_example_dropdown_change)

def on_solve_button_clicked(b):
    global gen
    global solution
    
    val = sudoku.value.copy()
    sudoku.fixed = [v > 0 for v in val]
    gen = solve_sudoku(val)
    try:
        solution = next(gen)
        sudoku.value = solution
    except StopIteration:
        label.value = "This sudoku has no solution."
        sudoku.fixed = [False] * 81
        return
    
    try:
        solution = next(gen).copy()
        label.value = "This sudoku has multiple solutions."
        solve_button.layout.display = 'none'
        next_button.layout.display = 'inline-block'
    except StopIteration:
        label.value = ""
        solve_button.layout.display = 'none'
    
solve_button.on_click(on_solve_button_clicked)

def on_next_button_clicked(b):
    global gen
    global solution
    
    sudoku.value = solution
    try:
        solution = next(gen)
    except StopIteration:
        label.value = ""
        next_button.layout.display = 'none'

next_button.on_click(on_next_button_clicked)

In [7]:
def solve_sudoku(puzzle, index=0):
    if index == 81:
        # Solution found
        yield puzzle
    elif puzzle[index] > 0:
        # Already filled
        yield from solve_sudoku(puzzle, index + 1)
    else:
        for v in range(1,10):
            # Fill in a digit and check constraints
            puzzle[index] = v
            if is_valid_square(puzzle, index):
                yield from solve_sudoku(puzzle, index + 1)
            puzzle[index] = 0

In [8]:
def get_column(puzzle, k):
    column = []
    for i in range(9):
        column.append(puzzle[i*9 + k])
    return column

def get_row(puzzle, r):
    return puzzle[r*9:(r+1)*9]

def get_block(puzzle, b):
    block = []
    for r in range(3):
        for k in range(3):
            block.append(puzzle[[0,3,6,27,30,33,54,57,60][b]+9*r+k])
    return block

def is_valid(l):
    # Check for duplicate values
    digits = [v for v in l if v > 0]
    s = set(digits)
    return len(digits) == len(s)

def is_valid_square(puzzle, i):
    k = i % 9
    r = int(i / 9)
    b = int(r / 3) * 3 + int(k / 3)
    
    return is_valid(get_row(puzzle, r)) and is_valid(get_column(puzzle, k)) and is_valid(get_block(puzzle, b))

In [9]:
display(hbox, label)

Label(value='')

Enter a sudoku puzzle to solve or select one of the three examples.